In [1]:
# 성능향상 기법
# 1. 파생피처 추가
# 2. 베이지안 최적화 

In [2]:
# 베이스 라인 모델로 Light BGM 모델을 사용할거임 
# 책에서는 파이썬 래퍼 LightGMB 과 사이킷런 LightGBM 중 파이썬 래퍼 LightBGM 사용

import os.path
from os import path
import pandas as pd

# 캐글 ↓제출경로
data_path = './input/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test  = pd.read_csv(data_path + 'test.csv' , index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) # 타깃값 제거 
all_features = all_data.columns

# 명목형 피처는 보통 웟-핫 인코딩 적용하는게 정석인듯 하다 
# 명목형 피처 추출
from sklearn.preprocessing import OneHotEncoder
cat_features = [feature for feature in all_features if 'cat' in feature]

# 원 핫 인코딩
onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

In [3]:
# 파생 피처 추가 
# 데이터 하나당 결측값 개수를 파생 피처로 추가

all_data['num_missing'] = (all_data==-1).sum(axis=1)

In [4]:
# 명목형 피처, calc 분류의 피처를 제외한 피처
remaining_features = [
    feature for feature in all_features
    if ('cat' not in feature and 'calc' not in feature)
]

# num_missing 을 remaining_features 에 추가
remaining_features.append('num_missing')

In [5]:
# 갑자기 뜬금없는 피처 엔지니어링 

ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature = True
for ind_featrue in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_featrue].astype(str) + '_'
        is_first_feature = False
    else:
        all_data['mix_ind'] += all_data[ind_featrue].astype(str) + '_'
        
all_data['mix_ind']

0          2_2_5_1_0_0_1_0_0_0_0_0_0_0_11_0_1_0_
1           1_1_7_0_0_0_0_1_0_0_0_0_0_0_3_0_0_1_
2          5_4_9_1_0_0_0_1_0_0_0_0_0_0_12_1_0_0_
3           0_1_2_0_0_1_0_0_0_0_0_0_0_0_8_1_0_0_
4           0_2_0_1_0_1_0_0_0_0_0_0_0_0_9_1_0_0_
                           ...                  
1488023     0_1_6_0_0_0_1_0_0_0_0_0_0_0_2_0_0_1_
1488024    5_3_5_1_0_0_0_1_0_0_0_0_0_0_11_1_0_0_
1488025     0_1_5_0_0_1_0_0_0_0_0_0_0_0_5_0_0_1_
1488026    6_1_5_1_0_0_0_0_1_0_0_0_0_0_13_1_0_0_
1488027    7_1_4_1_0_0_0_0_1_0_0_0_0_0_12_1_0_0_
Name: mix_ind, Length: 1488028, dtype: object

In [6]:
all_data['ps_ind_02_cat'].value_counts()

 1    1079327
 2     309747
 3      70172
 4      28259
-1        523
Name: ps_ind_02_cat, dtype: int64

In [7]:
all_data['ps_ind_02_cat'].value_counts().to_dict()

{1: 1079327, 2: 309747, 3: 70172, 4: 28259, -1: 523}

In [14]:
cat_count_features = []
for feature in cat_features + ['mix_ind']:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x: val_counts_dict[x])
    
    cat_count_features.append(f'{feature}_count')

In [11]:
cat_count_features

['ps_ind_02_cat_count',
 'ps_ind_04_cat_count',
 'ps_ind_05_cat_count',
 'ps_car_01_cat_count',
 'ps_car_02_cat_count',
 'ps_car_03_cat_count',
 'ps_car_04_cat_count',
 'ps_car_05_cat_count',
 'ps_car_06_cat_count',
 'ps_car_07_cat_count',
 'ps_car_08_cat_count',
 'ps_car_09_cat_count',
 'ps_car_10_cat_count',
 'ps_car_11_cat_count',
 'mix_ind_count']

In [15]:
all_data['mix_ind_count']

0             6
1            36
2            24
3          2784
4           258
           ... 
1488023     107
1488024      26
1488025     258
1488026      37
1488027      67
Name: mix_ind_count, Length: 1488028, dtype: int64

In [18]:
# 필요없다고 판단한 피처는 제거한 다음 모든 데이터 합치기 
from scipy import sparse 

# 필요 없는 피처들
drop_features = [
    'ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin',
    'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14'
]

# remaining_features, cat_count_features 에서 drop_features 를 제거한 데이터 
all_data_remaining = all_data[remaining_features + cat_count_features].drop(drop_features, axis=1)

# 데이터 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                               encoded_cat_matrix],
                               format='csr'
                               )

In [19]:
type(all_data_sprs)

scipy.sparse._csr.csr_matrix

In [20]:
# 데이터 나누기
num_train = len(train)

X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values 

In [21]:
import lightgbm as lgb 
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

# 베이지안 최적화 데이터셋
bayes_dtrain = lgb.Dataset(X_train, y_train)
bayes_dvalid = lgb.Dataset(X_valid, y_valid)

In [41]:
# 베이지안 최적화를 위한 하이퍼 파라미터 범위 설정 

param_bounds = {
    'num_leaves': (30,40), # 개별 트리가 가질수 있는 최대 말단 노드 개수
    'lambda_l1': (0.7, 0.9), # L1 규제 조정값 
    'lambda_l2': (0.9, 1), # L2 규제 조정값 
    'feature_fraction': (0.6, 0.7), # 개별 트리를 훈련할때 사용하는 피처 샘플링 비율 
    'bagging_fraction': (0.6, 0.9), # 개별 트리 훈련할때 사용할 데이터 샘플링 비율  
    'min_child_samples': (6, 10), # 말단 노드가 되기 위해 필요한 최소 데이터 갯수
    'min_child_weight': (10, 40) # 과대적합 방지를 위한 값 
}

# 값이 고정된 하이퍼 파라미터
fixed_params = {
    'objective': 'binary', # 이진분류라서 binary 설정
    'learning_rate': 0.005, # 학습률은 0.005
    'bagging_freq': 1, # 배깅 수행 빈도는 1 
    'force_row_wise': True,  # 경고문구를 없애기 위해 True 설정
    'random_state': 1991 # 설정 
}

In [42]:
import numpy as np

def eval_gini(y_true, y_pred):
    # 실젯값과 예측값의 크기가 서로 같은지 확인 (값이 다르면 오류 발생 )
    
    assert y_true.shape == y_pred.shape
    
    n_samples = y_true.shape[0] # 데이터 개수
    L_mid = np.linspace( 1/ n_samples, 1, n_samples) # 1/n_samples~ 1 까지 n_samples-1 개의 구간으로 나누겠다. 
    
    # 1) 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()]
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)
    
    # 2) 예측 완벽할때 지니계수
    true_order = y_true[y_true.argsort()]
    L_true = np.cumsum(true_order) / np.sum(true_order)
    G_true = np.sum(L_mid - L_true)
    
    # 정규화된 지니계수
    return G_pred / G_true

In [51]:
# LightGBM 용 gini() 함수
def gini(preds, dtrain):
    labels = dtrain.get_label() # get_label 은 데이터셋의 타깃값 반환
    return 'gini', eval_gini(labels, preds) , True # 반환값 

In [43]:
# (베이지안 최적화용) 평가지표 계산 함수 작성 

def eval_function(num_leaves , lambda_l1, lambda_l2, feature_fraction, 
                  bagging_fraction, min_child_samples, min_child_weight):
    '''최적화 하려는 평가지표(지니계수) 계산 함수 '''
    
    # 베이지안 최적화를 수행할 하이퍼 파라미터
    params = {
        'num_leaves': int(round(num_leaves)),
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight' : min_child_weight,
        'feature_pre_filter': False
    }
    
    params.update(fixed_params)
    
    print('하이퍼 파라미터 : ',params)
    
    # LightGBM 모델 훈련 
    
    lgb_model = lgb.train(
        params=params,
        train_set=bayes_dtrain,
        num_boost_round=2500,
        valid_sets=bayes_dvalid,
        verbose_eval=False
    )
    
    # 검증 데이터로 예측 수행 
    preds = lgb_model.predict(X_valid)
    # 지니계수 계산
    gini_score = eval_gini(y_valid, preds)
    print(f'지니계수 : {gini_score}\n')
    
    return gini_score

In [44]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성 
optimizer = BayesianOptimization(f=eval_function, # 평가지표 계산 함수
                                 pbounds=param_bounds, # 하이퍼 파라미터 범위
                                 random_state=0)

In [45]:
# 베이지안 최적화 수행
optimizer.maximize(init_points=3, n_iter=6)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
하이퍼 파라미터 :  {'num_leaves': 34, 'lambda_l1': 0.8205526752143287, 'lambda_l2': 0.9544883182996897, 'feature_fraction': 0.6715189366372419, 'bagging_fraction': 0.7646440511781974, 'min_child_samples': 8, 'min_child_weight': 29.376823391999682, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}


c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1555
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.28554700343777367

| 1         | 0.2855    | 0.7646    | 0.6715    | 0.8206    | 0.9545    | 7.695     | 29.38     | 34.38     |
하이퍼 파라미터 :  {'num_leaves': 39, 'lambda_l1': 0.7766883037651555, 'lambda_l2': 0.9791725038082665, 'feature_fraction': 0.6963662760501029, 'bagging_fraction': 0.867531900234624, 'min_child_samples': 8, 'min_child_weight': 27.04133683281797, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1555
[LightGBM] [Info] Number of data points i

c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 1555
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.2830327176874422

| 4         | 0.283     | 0.8978    | 0.6594    | 0.8445    | 0.9234    | 8.619     | 10.55     | 30.09     |
하이퍼 파라미터 :  {'num_leaves': 37, 'lambda_l1': 0.7738449330497988, 'lambda_l2': 0.9032695189818599, 'feature_fraction': 0.6606341064409726, 'bagging_fraction': 0.7666713964943057, 'min_child_samples': 9, 'min_child_weight': 29.306172421380474, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786


...

지니계수 : 0.2851552116496103

| 7         | 0.2852    | 0.6       | 0.6362    | 0.7207    | 0.9909    | 6.469     | 33.95     | 38.37     |
하이퍼 파라미터 :  {'num_leaves': 40, 'lambda_l1': 0.7672855841634896, 'lambda_l2': 0.9768876584971635, 'feature_fraction': 0.6519643254785416, 'bagging_fraction': 0.8892105163800896, 'min_child_samples': 6, 'min_child_weight': 39.17165367785253, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}


c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1555
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.28515214660631016

| 8         | 0.2852    | 0.8892    | 0.652     | 0.7673    | 0.9769    | 6.378     | 39.17     | 39.98     |
하이퍼 파라미터 :  {'num_leaves': 31, 'lambda_l1': 0.7666126427497625, 'lambda_l2': 0.953427422198884, 'feature_fraction': 0.6535635442606814, 'bagging_fraction': 0.8406992841576415, 'min_child_samples': 10, 'min_child_weight': 26.792694730461516, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1555
[LightGBM] [Info] Number of data points

c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.284430276342444

| 9         | 0.2844    | 0.8407    | 0.6536    | 0.7666    | 0.9534    | 9.675     | 26.79     | 31.26     |


In [46]:
max_params = optimizer.max['params']
max_params

{'bagging_fraction': 0.7646440511781974,
 'feature_fraction': 0.6715189366372419,
 'lambda_l1': 0.8205526752143287,
 'lambda_l2': 0.9544883182996897,
 'min_child_samples': 7.694619197355619,
 'min_child_weight': 29.376823391999682,
 'num_leaves': 34.37587211262692}

In [47]:
# num_leaves 와 min_child_samples 는 원래 정수형 하이퍼 파라미터이므로 정수형으로 변환하여 다시 저장 
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))

In [48]:
# 값이 고정된 하이퍼 파라미터 추가 
max_params.update(fixed_params)

In [49]:
max_params

{'bagging_fraction': 0.7646440511781974,
 'feature_fraction': 0.6715189366372419,
 'lambda_l1': 0.8205526752143287,
 'lambda_l2': 0.9544883182996897,
 'min_child_samples': 8,
 'min_child_weight': 29.376823391999682,
 'num_leaves': 34,
 'objective': 'binary',
 'learning_rate': 0.005,
 'bagging_freq': 1,
 'force_row_wise': True,
 'random_state': 1991}

In [50]:
# 모델 훈련 및 성능 검증  
# 6,7 장에서 사용한 그리드 서치와 달리 베이지안 최적화는 최적 예측기 (최적 하이퍼 파라미터 값들로 훈련된 모델)을 제공해
# 주지 않는다. 따라서 베이지안 최적화로 찾은 하이퍼 파라미터를 활용해 Light GBM 다시 훈련시켜야함 
# LightGBM 훈련시 사용한 파라미터만 max_params로 바꿨을 뿐임. 

In [52]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기 생성
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담은 1차원 배열 
oof_val_preds = np.zeros(X.shape[0])
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담은 1차원 배열 
oof_test_preds = np.zeros(X_test.shape[0])

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정 
    
    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터
    
    # LightGBM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train, y_train) # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid) # LightGBM 전용 검증 데이터셋
    
    # LightGBM 모델 훈련 
    lgb_model = lgb.train(params=max_params, # 최적 하이퍼 파라미터
                          train_set=dtrain, # 훈련 데이터셋
                          num_boost_round=2500, # 부스팅 반복 횟수
                          valid_sets=dvalid, # 성능 평가용 검증 데이터셋
                          feval=gini, # 검증용 평가지표
                          early_stopping_rounds=300, # 조기종료 조건
                          verbose_eval=100) # 100 번쨰 마다 점수 출력
    
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 정규화 지니계수
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수 : {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1554
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 216
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 300 rounds
[100]	valid_0's binary_logloss: 0.154282	valid_0's gini: 0.266871
[200]	valid_0's binary_logloss: 0.153223	valid_0's gini: 0.273142
[300]	valid_0's binary_logloss: 0.152631	valid_0's gini: 0.278209
[400]	valid_0's binary_logloss: 0.152272	valid_0's gini: 0.281871
[500]	valid_0's binary_logloss: 0.15203	valid_0's gini: 0.285298
[600]	valid_0's binary_logloss: 0.151872	valid_0's gini: 0.287682
[700]	valid_0's binary_logloss: 0.151758	valid_0's gini: 0.289711
[800]	valid_0's binary_logloss: 0.151669	valid_0's gini: 0.291441
[900]	valid_0's binary_logloss: 0.151605	valid_0's gini: 0.292794
[1000]	val

In [53]:
print('OOF 검증 데이터 지니계수 : ',eval_gini(y, oof_val_preds))

OOF 검증 데이터 지니계수 :  0.28863294839417913


In [54]:
submission['target'] = oof_test_preds
submission.to_csv('./output/submission_2.csv')

In [55]:
!kaggle competitions submit -c porto-seguro-safe-driver-prediction -f ./output/submission_2.csv -m submit_in_local

Successfully submitted to Porto Seguro’s Safe Driver Prediction



  0%|          | 0.00/24.4M [00:00<?, ?B/s]
  0%|          | 8.00k/24.4M [00:00<13:52, 30.7kB/s]
  0%|          | 112k/24.4M [00:00<01:06, 383kB/s]  
  4%|▎         | 904k/24.4M [00:00<00:08, 2.86MB/s]
  8%|▊         | 1.92M/24.4M [00:01<00:12, 1.94MB/s]
 17%|█▋        | 4.10M/24.4M [00:01<00:04, 4.82MB/s]
 24%|██▍       | 5.84M/24.4M [00:01<00:02, 7.03MB/s]
 35%|███▌      | 8.60M/24.4M [00:01<00:01, 11.2MB/s]
 42%|████▏     | 10.3M/24.4M [00:02<00:02, 5.50MB/s]
 52%|█████▏    | 12.7M/24.4M [00:02<00:01, 7.67MB/s]
 58%|█████▊    | 14.2M/24.4M [00:02<00:02, 4.99MB/s]
 65%|██████▍   | 15.8M/24.4M [00:03<00:01, 6.23MB/s]
 70%|███████   | 17.1M/24.4M [00:03<00:01, 6.18MB/s]
 74%|███████▍  | 18.1M/24.4M [00:03<00:01, 6.05MB/s]
 78%|███████▊  | 19.1M/24.4M [00:03<00:00, 6.09MB/s]
 81%|████████▏ | 19.9M/24.4M [00:03<00:00, 5.93MB/s]
 84%|████████▍ | 20.6M/24.4M [00:03<00:00, 6.04MB/s]
 87%|████████▋ | 21.3M/24.4M [00:03<00:00, 6.06MB/s]
 90%|█████████ | 22.0M/24.4M [00:04<00:00, 5.94MB/s]
 9